# Import Libraries

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from sqlalchemy import create_engine
import dateutil
import isodate
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
load_dotenv()
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

# Guideline Notes

There are many steps to making data analysis for this project, which is about making insights about the activity and performance of Youtube creators from the Charon Company in South Korea. Here are some of the guideline notes for this notebook specifically.   
For more details refer [README.md](./README.md)...

1. Gather necessary Youtube Data for every Charon Creator for Data Analysis. (`Python`)
2. Connect data to PostgreSQL. (`PostgreSQL`)
3. Join necessary tables for Data Analysis. (`SQL`)
4. Perform Exploratory Data Analysis for insights. (`Pandas`)
5. Data Wrangling. (`Pandas`)
6. Make necessary queries for visuals (plots, charts, etc.). (`SQL`/`Tableau`)
7. Perform Hypothesis Testing for findings from queries. (`Python`)
8. Create Dashboard in Tableau. (`Tableau`)

# Exploratory Data Analysis

In the EDA process the data will be studied to see if there are any missing values that are worth cleaning or changing, features that should be formatted differently, get insights to see if we can remove or add some additional features (feature engineering) and more.  

In this process:
1. A connection to the database will be made to delve into the data.
1. The data will be explored to see if there are any missing values that are worth cleaning.
2. Get insights to changes that should be made during the Data Wrangling process.

In [3]:
# Database connection parameters
db_params = {
    "host":"127.0.0.1",
    "database":"charon-analysis",
    "user":"postgres",
    "password":os.environ.get('PASSWORD'),
    "port":"5432"
}

# Create a connection to the PostgreSQL server
conn = psycopg2.connect(
    host=db_params['host'],
    database=db_params['database'],
    user=db_params['user'],
    password=db_params['password']
)

# Create SQLAlchemy Engine with necessary URI
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

In [4]:
df = pd.read_sql_query(
"""
SELECT
	a.name
	,b.channel_name
	,b.created_at
	,b.country
	,b.view_count AS channel_view_count
	,b.subscriber_count
	,b.video_count
	,d.published_at
	,d.title
	,d.description
	,d.thumbnails
	,d.tags
	,d.category_id
	,e.category_name
	,d.default_audio_language
	,d.duration
	,d.view_count AS video_view_count
	,d.like_count
	,d.comment_count
FROM creator_channel a
FULL OUTER JOIN channel_stats b
	ON a.channel_id = b.channel_id
FULL OUTER JOIN playlist_video c
	ON b.playlist_id = c.playlist_id
FULL OUTER JOIN video_stats d
	ON c.video_id = d.video_id
INNER JOIN category_id_name e
	ON d.category_id = e.category_id
;
"""
, con=engine)
df[2000:2005]

,name,channel_name,created_at,country,channel_view_count,subscriber_count,video_count,published_at,title,description,thumbnails,tags,category_id,category_name,default_audio_language,duration,video_view_count,like_count,comment_count
2000,아빠킹,아빠킹,2011-07-15T06:19:52Z,KR,405032510,314000,4647,2018-03-25T00:51:50Z,[철권7] 아직...나는 만족하지 못했다.,아빠킹의 생방송 주소 : https://www.twitch.tv/rldnddl789...,https://i.ytimg.com/vi/hoEz-HiVjSA/default.jpg,"['아빠킹', '철권', '철권7', 'tekken', 'tekken7', '대전격...",20,Gaming,None,PT18M42S,49609,317.0,150.0
2001,아빠킹,아빠킹,2011-07-15T06:19:52Z,KR,405032510,314000,4647,2018-03-24T08:46:30Z,[철권7] 빠티스 멈추지않는 질주,아빠킹의 생방송 주소 : https://www.twitch.tv/rldnddl789...,https://i.ytimg.com/vi/SaDYoHwewVQ/default.jpg,"['아빠킹', '철권', '철권7', 'tekken', 'tekken7', '대전격...",20,Gaming,None,PT22M20S,56774,299.0,169.0
2002,아빠킹,아빠킹,2011-07-15T06:19:52Z,KR,405032510,314000,4647,2018-03-23T16:51:44Z,[철권7] 약빨고 하는 데스매치 ㅋㅋㅋ,아빠킹의 생방송 주소 : https://www.twitch.tv/rldnddl789...,https://i.ytimg.com/vi/K-hZkCqHv4U/default.jpg,"['아빠킹', '철권', '철권7', 'Tekken', 'Tekken7', '대전격...",20,Gaming,None,PT18M14S,24156,138.0,32.0
2003,아빠킹,아빠킹,2011-07-15T06:19:52Z,KR,405032510,314000,4647,2018-03-23T09:24:36Z,[아빠킹] 녹티스 하루만에 왕좌에 앉다.,아빠킹의 생방송 주소 : https://www.twitch.tv/rldnddl789...,https://i.ytimg.com/vi/BtHVE-pATEo/default.jpg,"['아빠킹', '철권', '철권7', 'tekken', 'tekken7', '대전격...",20,Gaming,None,PT21M7S,97037,466.0,139.0
2004,아빠킹,아빠킹,2011-07-15T06:19:52Z,KR,405032510,314000,4647,2018-03-23T02:50:35Z,[철권7] 무패행진 빠티스 어디까지?,아빠킹의 생방송 주소 : https://www.twitch.tv/rldnddl789...,https://i.ytimg.com/vi/gAOqhW7EWl4/default.jpg,"['아빠킹', '철권', '철권7', 'tekken', 'tekken7', '대전격...",20,Gaming,None,PT27M51S,34293,191.0,76.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98593 entries, 0 to 98592
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    98593 non-null  object 
 1   channel_name            98593 non-null  object 
 2   created_at              98593 non-null  object 
 3   country                 95339 non-null  object 
 4   channel_view_count      98593 non-null  int64  
 5   subscriber_count        98593 non-null  int64  
 6   video_count             98593 non-null  int64  
 7   published_at            98593 non-null  object 
 8   title                   98593 non-null  object 
 9   description             94176 non-null  object 
 10  thumbnails              98593 non-null  object 
 11  tags                    85113 non-null  object 
 12  category_id             98593 non-null  int64  
 13  category_name           98593 non-null  object 
 14  default_audio_language  68908 non-null

In [6]:
df.isna().sum()

name                          0
channel_name                  0
created_at                    0
country                    3254
channel_view_count            0
subscriber_count              0
video_count                   0
published_at                  0
title                         0
description                4417
thumbnails                    0
tags                      13480
category_id                   0
category_name                 0
default_audio_language    29685
duration                      0
video_view_count              0
like_count                 4548
comment_count              1065
dtype: int64

In [7]:
df.describe()

,channel_view_count,subscriber_count,video_count,category_id,video_view_count,like_count,comment_count
count,9.859300e+04,9.859300e+04,98593.000000,98593.000000,9.859300e+04,94045.000000,97528.000000
mean,2.520061e+08,2.580834e+05,4457.058736,19.646385,1.069176e+05,1367.802307,121.487337
std,2.473276e+08,2.312868e+05,3834.423424,2.973859,2.544682e+05,4314.589439,366.032051
min,3.840600e+04,2.200000e+02,18.000000,1.000000,0.000000e+00,0.000000,0.000000
25%,5.911680e+07,6.850000e+04,1478.000000,20.000000,3.689000e+03,28.000000,6.000000
50%,1.512856e+08,1.830000e+05,3484.000000,20.000000,2.809100e+04,192.000000,40.000000
75%,4.050325e+08,4.320000e+05,6038.000000,20.000000,1.086230e+05,909.000000,123.000000
max,1.308509e+09,1.240000e+06,12551.000000,28.000000,9.395104e+06,234165.000000,45297.000000


# Data Wrangling

Changes needed are:
1. Change created_at, published_at from ISO 8601 UTC(Coordinated Universal Time) to KST(Korean Standard Time).
2. Add a feature to show what day of the week the video was uploaded.
3. Add a feature to show the ISO 8601 duration in total amount of seconds.
4. Change duration by removing the P and T for better readability (P stands for Period and T stands for Time).
5. Change null values in like_count and comment_count to 0 and to int64 / bigint.
6. It seems that some creators from Charon are not active in youtube anymore maybe because they are targetting other SNS platforms so exclude those from analysis, to be more specific, include only creators who have been active in the last 6 months (due to policy regarding payments).
7. Apply all the changes above to make a final DataFrame.

In [8]:
# 1. Change created_at, published_at from ISO 8601 UTC(Coordinated Universal Time) to KST(Korean Standard Time)
df['created_at'] = df['created_at'].apply(lambda x: dateutil.parser.parse(x))
df['created_at'] = df['created_at'].dt.tz_convert('Asia/Seoul').dt.tz_localize(None)
df['published_at'] = df['published_at'].apply(lambda x: dateutil.parser.parse(x))
df['published_at'] = df['published_at'].dt.tz_convert('Asia/Seoul').dt.tz_localize(None)

In [9]:
# 2. Add a feature to show what day of the week the video was uploaded.
df['published_day'] = df['published_at'].apply(lambda x: x.strftime('%A'))

In [10]:
# 3. Add a feature to show the ISO 8601 duration in total amount of seconds.
df['duration_seconds'] = df['duration'].apply(lambda x: isodate.parse_duration(x).total_seconds())


In [11]:
# 4. Change duration by removing the P and T for better readability (P stands for Period and T stands for Time).
df['duration'] = df['duration'].apply(lambda x: x.replace('P', '').replace('T', ''))

In [12]:
# 5. Change like_count and comment_count to in64 / bigint.
df['like_count'] = df['like_count'].fillna(0).round().astype('int64')
df['comment_count'] = df['comment_count'].fillna(0).round().astype('int64')

In [13]:
# 6. It seems that some creators from Charon are not active in youtube anymore maybe because they are targetting other SNS platforms so exclude those from analysis,
# to be more specific, include only creators who have been active in the last 6 months (due to policy regarding payments).

max_date = df['published_at'].dt.date.max()
cutoff_date = max_date - pd.DateOffset(months=6)
active_channel_mask = df['published_at'] >= cutoff_date
df_active = df[active_channel_mask]
active_channels = list(df_active['name'].unique())
print(f"Current Active Channels: {active_channels}")
print(f"Total of: {len(active_channels)} channels.")

Current Active Channels: ['지명', '송사리', '아빠킹', '삐야기', '정동글', '흐쟁이', '신해조', '학살', '스나랑', '인섹', '강인경', '미야', '도개', '이재석', '이클리피아', '링규링규링', '블루위키', 'THIRD', '모양몬', '핑맨', '김용녀', '추털이', '제갈병춘', '전쓰트', '프로젝트롤', '후즈', '피닉스박', 'ATK', '김데데', '꼬예유', 'PAKA', '소행성612', '한동숙', '지보배', '괴물쥐', '명훈', '헤이스트', '냄새', '이석현', '코뚱잉', '모아요', '죠니월드', '미남홀란드', '랄로', '류제홍', '스틸로', '막눈', '버니버니', '른쿄', '멀럭킹', '돌카사', '꼴랑이', '순규박', '2수연', '핑크자크', '종원TV', '고수달', '또시', '윤가놈']
Total of: 59 channels.


In [14]:
# 7. Apply all the changes above to make a final DataFrame.
df_final = df.loc[df['name'].isin(active_channels)]
df_final.head()

,name,channel_name,created_at,country,channel_view_count,subscriber_count,video_count,published_at,title,description,...,tags,category_id,category_name,default_audio_language,duration,video_view_count,like_count,comment_count,published_day,duration_seconds
669,지명,지명,2014-06-05 06:59:37,KR,11988014,37700,199,2024-05-16 16:24:42,"『검밑솔돌이』, 과연 효율적일까? 비용과 본캐 투자 비교까지 완전 분석! 【메이플】",#지명 #메이플 #검밑솔 지명 치지직 : https://chzzk.naver.co...,...,"['메이플', '메이플 21주년', '메이플 이벤트', '메이플 이벤트 피드백', ...",20,Gaming,ko,12M21S,36451,243,258,Thursday,741.0
670,지명,지명,2014-06-05 06:59:37,KR,11988014,37700,199,2024-05-12 21:00:20,"18성 둘둘로 세칼카 솔플하기! 실패 시 벌칙까지..? 【메이플, 카데나 EP.1】",#지명 #메이플 #카데나 #18성둘둘 지명 치지직 : https://chzzk.n...,...,"['메이플', '메이플 21주년', '메이플 이벤트', '메이플 이벤트 피드백']",20,Gaming,ko,9M20S,17756,116,25,Sunday,560.0
671,지명,지명,2014-06-05 06:59:37,KR,11988014,37700,199,2024-04-18 10:00:00,드디어 시작되는 21주년 이벤트 완전 분석! 그리고 효율적으로 즐기는 방법! 【메이플】,#지명 #메이플 #21주년 지명 치지직 : https://chzzk.naver.c...,...,"['메이플', '메이플 21주년', '메이플 이벤트', '메이플 이벤트 피드백']",20,Gaming,ko,13M50S,37359,234,63,Thursday,830.0
672,지명,지명,2014-06-05 06:59:37,KR,11988014,37700,199,2024-03-15 18:00:48,널뛰기로 느끼는 사람 【메이플】,#지명 #메이플스토리 2024년 2월 2일 생방송 다시보기입니다. 해당 다시보기 ...,...,None,20,Gaming,None,1M,2830,34,12,Friday,60.0
673,지명,지명,2014-06-05 06:59:37,KR,11988014,37700,199,2024-03-07 18:00:45,데벤 6차가 빨간 이유 【메이플】,#지명 #메이플스토리 2024년 1월 18일 다시보기 하이라이트입니다. 해당 다시...,...,None,20,Gaming,None,43S,4750,60,11,Thursday,43.0


# Export To PostgreSQL

Export Data to PostgreSQL for Data Analysis

In [15]:
# df_final.to_csv('./data/charon_data.csv', index=False)

In [16]:
# # Define the file paths for your CSV files
# csv_files = {
#     'charon_data': './data/charon_data.csv',
# }

# # Load and display the contents of each CSV file to check
# for table_name, file_path in csv_files.items():
#     print(f"Contents of '{table_name}' CSV file:")
#     df = pd.read_csv(file_path)
#     print(df.head(2))  # Display the first few rows of the DataFrame
#     print("\n")
    
# # Loop through the CSV files and import them into PostgreSQL
# for table_name, file_path in csv_files.items():
#     df = pd.read_csv(file_path)
#     df.to_sql(table_name, engine, if_exists='replace', index=False)

# Data Analysis

In the Data Analysis process the preprocessed data will be used to make insights to what the data is trying to say. This will involve answering questions through descriptive statistics and inferential statistics.

This involves specifying dependent and independent variables. According to Youtube's Monetary Policy, there are various ways to generate money but the main way and constant is through views because other ways such as putting advertisements in between videos is also dependent on views. That is why all types of metrics will be based on views as the independent variable.

The following questions are from brainstorming for different types of statistics that I thought would be good to try and observe, from which I tried to make SQL queries, if possible, to find some patterns detectable to the human eye. [refer to data_analysis.sql](./data_analysis.sql)  

`Descriptive Statistics`
- Who in Charon has the biggest channel in terms of subscription? in terms of total views?
- Who has the most views per video?
- Which are the most watched and least watched videos for each creator?
- Does covering a wider range of category improve total views of the channel?
- Does the upload period of the year, month, day, time matter?
- What category is the most trending?
- How does advertisement videos compare to non-advertisement videos on views performance?
- Is there a drop or increase in views to consecutive videos after advertisement?
- Is the length of the video related to views? Does it have a different effect in different categories?
- Is there a relationship between how frequent the uploads are? or how consistent the upload times are?
- Does the visuals of the creator being face, no-face, or virtual matter?
- Does having an intro or outro matter?
- Does having subtitles or no-subtitles matter? (subtitles of same language)
- Does the length of the title show a difference in views performance?
- Does likes and comments have a relationship with views?
- How big of an impact is there on amount of subscribers and views?  

After trying to query for these questions...

`Queries that will be used for Descriptive Statistics`  

1. Who in Charon has the biggest channel in terms of subscription? in terms of total views?
2. Which are the most watched and least watched videos for each creator?
3. Does covering a wider range of category improve total views of the channel?
4. Does the upload period of the year, month, day, time matter?
5. What category is the most trending?
6. Is the length of the video related to views? Does it have a different effect in different categories?
7. Does the visuals of the creator being face, no-face, or virtual matter?
8. Does the length of the title show a difference in views performance?  

`Queries that will be used for Inferential Statistics (Hypothesis Testing)`  

1. Does the upload time matter? (4.2)
2. What category is the most trending? (5)
3. Is the length of the video related to views? Shorts vs. Videos (6.1)
4. Does it have a different effect in different categories? (6.2)
5. From only normal videos (not shorts), which types of video   lengths have the most views? (6.3)
6. From only normal videos (not shorts), which types of video lengths have the most views by category? (6.4)  
7. Does the visuals of the creator being face, no-face, or virtual matter?  
8. Does the length of the title show a difference in views performance?  


In [17]:
df = pd.read_csv('./data/charon_data.csv')

In [18]:
def sql_query(query, con=engine):
    df = pd.read_sql(query, con)
    return df

### Queries for Descriptive Stats

1. Who in Charon has the biggest channel in terms of subscription? in terms of total views?

In [19]:
# 1.1 Order Creators from Most to Least Subscribers
df_1 = sql_query(
"""
SELECT
	name
	,MAX(subscriber_count) AS subscriber_count
FROM charon_data_analysis
GROUP BY name
ORDER BY subscriber_count DESC;
"""   
)
df_1.head()

,name,subscriber_count
0,랄로,1240000
1,괴물쥐,1140000
2,PAKA,745000
3,삐야기,586000
4,전쓰트,557000


In [58]:
# 1.2 Order Creaters from Most to Least Views
df_2 = sql_query(
"""
SELECT
	name
	,MAX(subscriber_count) AS subscriber_count
	,MAX(channel_view_count) AS total_views
	,ROUND(AVG(video_view_count),2) AS avg_views_per_video
FROM charon_data_analysis
GROUP BY name
ORDER BY avg_views_per_video DESC;
"""
)
df_2.head()

,name,subscriber_count,total_views,avg_views_per_video
0,랄로,1240000,751282116,1465619.62
1,괴물쥐,1140000,1308509163,809671.25
2,PAKA,745000,650136628,765685.42
3,모아요,454000,243584638,509630.22
4,제갈병춘,122000,53417533,361041.11


2. Which are the most watched and least watched videos for each creator?

In [21]:
# 2.1 Which are the most watched videos for each creator?
df_3 = sql_query(
"""
WITH ranked_views AS (
SELECT
	name
	,title
	,video_view_count AS views
	,RANK() OVER(PARTITION BY name ORDER BY video_view_count DESC) AS rank_top
FROM charon_data_analysis
WHERE video_view_count <> 0
)
SELECT 
	name
	,views
	,title
FROM ranked_views
WHERE rank_top <= 5
"""
)
df_3.head()

,name,views,title
0,2수연,202532,많은 원딜러분들이 놓치고 있는 것들 (실제프로팀코치님께 들었던 원딜피드백)
1,2수연,88601,"[경험담] 원딜 티어별 점수 올리는 방법, 공격적인 원딜과 그렇지 않은 원딜의 차이"
2,2수연,77484,룰루서폿 이것만 잘해도 반은 먹고 들어갑니다
3,2수연,54958,2분루시안콤보
4,2수연,42357,신박한 케이틀린 잡기술 #Shorts


In [22]:
# 2.1 Which are the least watched videos for each creator?
df_4 = sql_query(
"""
WITH ranked_views AS (
SELECT
	name
	,title
	,video_view_count AS views
	,RANK() OVER(PARTITION BY name ORDER BY video_view_count) AS rank_bot
FROM charon_data_analysis
WHERE video_view_count <> 0
)
SELECT 
	name
	,views
	,title
FROM ranked_views
WHERE rank_bot <= 5
"""
)
df_4.head()

,name,views,title
0,2수연,492,【룰루 관전강의 2부】길지만 진짜 중요한 내용 !!! 시야잡는법 운영 등등 자세한 설명
1,2수연,580,같이 봐요 ㅎㅎ 2년전 영상..! (서폿유저 동기부여 영상)
2,2수연,790,오랜만에 하는 트위치 원딜 (누누서폿과 함께 로밍가기)
3,2수연,963,마스터 원딜러의 셀프 피드백 [징크스]
4,2수연,1000,【룰루 관전강의 1부】길지만 진짜 중요한 내용 !!! 시야잡는법 운영 등등 자세한 설명


3. Does covering a wider range of category improve total views of the channel?

In [57]:
df_5 = sql_query(
"""
WITH category_count_per_channel AS (
SELECT
	name
	,category_name
	,COUNT(*) 
FROM charon_data_analysis
GROUP BY name, category_name
), avg_view_per_channel AS (
SELECT
	name
	,ROUND(AVG(video_view_count),2) AS avg_view_count
FROM charon_data_analysis
GROUP BY name
)
SELECT 
	a.name
	,COUNT(category_name) AS category_count
	,MAX(avg_view_count) AS avg_view_count
FROM category_count_per_channel a
JOIN avg_view_per_channel b
	ON a.name = b.name
GROUP BY a.name
ORDER BY category_count DESC;
"""
)
df_5.head()

,name,category_count,avg_view_count
0,스틸로,10,85751.50
1,신해조,9,182931.83
2,코뚱잉,9,214739.34
3,PAKA,8,765685.42
4,피닉스박,7,108710.12


4. Does the upload period of the year, month, day, time matter?

In [24]:
# 4.1 Does the upload day matter? 
# Order of most viewed by day of the week for each Creator
df_6 = sql_query(
"""
SELECT
	name
	,published_day
	,ROUND(AVG(video_view_count),2) AS avg_view_count
FROM charon_data_analysis
GROUP BY name, published_day
ORDER BY name, avg_view_count DESC;
"""
)
df_6.head()

,name,published_day,avg_view_count
0,2수연,Tuesday,7766.33
1,2수연,Friday,6877.49
2,2수연,Monday,6736.19
3,2수연,Sunday,6707.64
4,2수연,Wednesday,6594.40


In [25]:
# 4.1 Does the upload day matter?
# Order of most viewed by day of the week overall
df_7 = sql_query(
"""
SELECT
	published_day
	,ROUND(AVG(video_view_count),2) AS avg_view_count
FROM charon_data_analysis
GROUP BY published_day
ORDER BY avg_view_count DESC;
"""
)
df_7.head()

,published_day,avg_view_count
0,Sunday,121418.69
1,Saturday,120027.07
2,Tuesday,112756.10
3,Wednesday,109726.32
4,Friday,109188.08


In [26]:
# 4.2 Does the upload time matter?
df_8 = sql_query(
"""
WITH time_separation AS (
SELECT
	name
	,published_at::time AS published_at
	,CASE
        WHEN published_at::time BETWEEN '00:00:00' AND '01:59:59' THEN 1
        WHEN published_at::time BETWEEN '02:00:00' AND '03:59:59' THEN 2
        WHEN published_at::time BETWEEN '04:00:00' AND '05:59:59' THEN 3
        WHEN published_at::time BETWEEN '06:00:00' AND '07:59:59' THEN 4
        WHEN published_at::time BETWEEN '08:00:00' AND '09:59:59' THEN 5
        WHEN published_at::time BETWEEN '10:00:00' AND '11:59:59' THEN 6
        WHEN published_at::time BETWEEN '12:00:00' AND '13:59:59' THEN 7
        WHEN published_at::time BETWEEN '14:00:00' AND '15:59:59' THEN 8
        WHEN published_at::time BETWEEN '16:00:00' AND '17:59:59' THEN 9
        WHEN published_at::time BETWEEN '18:00:00' AND '19:59:59' THEN 10
        WHEN published_at::time BETWEEN '20:00:00' AND '21:59:59' THEN 11
        WHEN published_at::time BETWEEN '22:00:00' AND '23:59:59' THEN 12
    END AS time_cat
	,video_view_count
FROM charon_data_analysis
)
SELECT
	time_cat
	,ROUND(AVG(video_view_count),2) AS avg_view_count
FROM time_separation
GROUP BY time_cat
ORDER BY avg_view_count DESC;
"""
)
df_8.head()

,time_cat,avg_view_count
0,12,224050.00
1,7,126331.25
2,9,119395.85
3,10,116873.85
4,11,107916.94


5. What category is the most trending?

In [59]:
df_9 = sql_query(
"""
WITH count_per_category AS (
SELECT
	category_name
	,COUNT(*) AS category_count
FROM charon_data_analysis
GROUP BY category_name
),
views_per_category AS (
SELECT
	category_name
	,ROUND(AVG(video_view_count),2) AS avg_view_count
FROM charon_data_analysis a
GROUP BY category_name
ORDER BY avg_view_count DESC
)
SELECT
	a.category_name
	,category_count
	,avg_view_count
FROM count_per_category a
JOIN views_per_category b
	ON a.category_name = b.category_name
ORDER BY avg_view_count DESC;
"""
)
df_9.head()

,category_name,category_count,avg_view_count
0,Film & Animation,164,402463.92
1,Music,20,330440.15
2,Sports,64,270685.94
3,Entertainment,892,243633.01
4,Science & Technology,13,242336.62


6. Is the length of the video related to views? Does it have a different effect in different categories?

In [28]:
# 6.1 Is the length of the video related to views? Shorts vs. Videos
df_10 = sql_query(
"""
SELECT 
	CASE
		WHEN duration_seconds <= 60 THEN 'shorts'
		WHEN duration_seconds > 60 THEN 'video'
	END AS content_type
	,ROUND(AVG(video_view_count),2) AS avg_view_count
FROM charon_data_analysis
GROUP BY content_type
ORDER BY avg_view_count;
"""
)
df_10.head()

,content_type,avg_view_count
0,video,101050.02
1,shorts,248237.87


In [29]:
# 6.2 Does it have a different effect on different categories?
df_11 = sql_query(
"""
SELECT 
	CASE
		WHEN duration_seconds <= 60 THEN 'shorts'
		WHEN duration_seconds > 60 THEN 'video'
	END AS content_type
	,category_name
	,ROUND(AVG(video_view_count),2) AS avg_view_count
FROM charon_data_analysis
GROUP BY content_type, category_name
ORDER BY content_type, avg_view_count DESC;
"""
)
df_11.head()

,content_type,category_name,avg_view_count
0,shorts,Sports,609407.17
1,shorts,Music,508857.00
2,shorts,Comedy,440451.95
3,shorts,Gaming,250946.28
4,shorts,Entertainment,223477.98


In [30]:
# 6.3 From only normal videos (not shorts), which types of video lengths have the most views?
df_12 = sql_query(
"""
WITH video_length_category AS (
SELECT
        category_name,
        CASE
            WHEN duration_seconds BETWEEN 60 AND 60*5 THEN 'short'
            WHEN duration_seconds BETWEEN 60*5+1 AND 60*20 THEN 'medium'
            WHEN duration_seconds > 60*20 THEN 'long'
        END AS video_length,
        video_view_count
    FROM charon_data_analysis
    WHERE duration_seconds > 60
)
SELECT
    video_length,
    ROUND(AVG(video_view_count), 2) AS avg_view_count
FROM video_length_category
GROUP BY video_length
ORDER BY avg_view_count DESC;
"""
)
df_12.head()

,video_length,avg_view_count
0,medium,133612.74
1,short,69117.95
2,long,68716.84


In [48]:
# 6.4 From only normal videos (not shorts), which types of video lengths have the most views by category?
df_13 = sql_query(
"""
WITH video_length_category AS (
SELECT
        category_name
        ,CASE
            WHEN duration_seconds BETWEEN 60 AND 60*5 THEN 'short'
            WHEN duration_seconds BETWEEN 60*5+1 AND 60*20 THEN 'medium'
            WHEN duration_seconds > 60*20 THEN 'long'
        END AS video_length,
        video_view_count
    FROM charon_data_analysis
    WHERE duration_seconds > 60
)
SELECT
    category_name,
    video_length,
    ROUND(AVG(video_view_count), 2) AS avg_view_count
FROM video_length_category
GROUP BY category_name, video_length
ORDER BY video_length DESC, avg_view_count DESC;
"""
)
df_13

,category_name,video_length,avg_view_count
0,Film & Animation,short,323899.00
1,Comedy,short,286777.91
2,Entertainment,short,203888.35
3,Howto & Style,short,159493.00
4,Sports,short,106752.23
5,Travel & Events,short,100635.00
6,Music,short,71944.67
7,People & Blogs,short,71247.83
8,Pets & Animals,short,67363.92
9,Gaming,short,65362.02


7. Does the visuals of the creator being face, no-face, or virtual matter?

In [32]:
df_14 = sql_query(
"""
WITH creator_category AS (
SELECT
	name
	,CASE
		WHEN name IN ('피닉스박', '죠니월드', '또시', '이재석', '모아요', '류제홍', '코뚱잉', '인섹', '김용녀', '돌카사', '학살', '막눈', '강인경', '한동숙', '꼴랑이', '윤가놈', '스틸로', '명훈', '랄로', '괴물쥐', '소행성612', '핑크자크', '버니버니', '신해조', '이석현') THEN 'face'
		WHEN name IN ('삐야기', '스나랑', '후즈') THEN 'virtual'
		WHEN name IN ('냄새', '링규링규링', '지명', '2수연', '전쓰트', '정동글', '프로젝트롤', '헤이스트', '른쿄', '블루위키', '아빠킹', '김데데', '도개', 'THIRD', '미남홀란드', '종원TV', '꼬예유', 'ATK', '추털이', '순규박', '모양몬', '이클리피아', '멀럭킹', '송사리', '제갈병춘', 'PAKA', '지보배', '핑맨', '흐쟁이', '미야', '고수달') THEN 'no-face'
	END AS creator_type
	,subscriber_count
	,channel_view_count
	,video_view_count
	,video_count
FROM charon_data_analysis
)
SELECT
	creator_type
	,ROUND(AVG(video_view_count),2) AS avg_view_count
	,ROUND(AVG(subscriber_count),2) AS avg_subscriber_count
FROM creator_category
GROUP BY creator_type;
"""
)
df_14.head()

,creator_type,avg_view_count,avg_subscriber_count
0,face,141890.96,352353.01
1,no-face,87619.74,193507.71
2,virtual,160586.84,488298.57


8. Does the length of the title show a difference in views performance?

In [33]:
df_15 = sql_query(
"""
WITH title_length_category AS (
SELECT
	CASE
		WHEN LENGTH(title) BETWEEN 0 AND 4 THEN 'short'
		WHEN LENGTH(title) BETWEEN 5 AND 20 THEN 'medium'
		WHEN LENGTH(title) BETWEEN 21 AND 35 THEN 'long'
		WHEN LENGTH(title) >= 36 THEN 'very long'
	END AS title_length
	,video_view_count AS view_count
FROM charon_data_analysis
)
SELECT
	title_length
	,ROUND(AVG(view_count),2) AS avg_view_count
FROM title_length_category
GROUP BY title_length
ORDER BY avg_view_count DESC;
"""
)
df_15.head()

,title_length,avg_view_count
0,short,312187.77
1,medium,213746.31
2,long,99908.36
3,very long,71083.08


### Queries for Inferential Stats

In this sections the queries that somewhat seem like it has a strong correlation only by eye will be analyzed with hypothesis testing to see if it is statistically significant.

**4.2 Does the upload time matter?**  

**Choice of Test**: ANOVA, Post-hoc (Tukey's HSD)  
ANOVA test will be used to see if there are significant differences between the time frames. The time frames are categories into 12 bins of 2-hour intervals, e.g. 1 is from 00:00:00 to 01:59:59 and 2 is from 02:00:00 to 03:59:59, etc.  

Then, Post-hoc Testing will be used to further analyze to determine which specific categories differ from each other and see if what can be seen from the eye that the category 12 holds significantly more views than other time window frame categories.

In [34]:
df_8_ht = sql_query(
"""
SELECT 
    *
    ,CASE
        WHEN published_at::time BETWEEN '00:00:00' AND '01:59:59' THEN 1
        WHEN published_at::time BETWEEN '02:00:00' AND '03:59:59' THEN 2
        WHEN published_at::time BETWEEN '04:00:00' AND '05:59:59' THEN 3
        WHEN published_at::time BETWEEN '06:00:00' AND '07:59:59' THEN 4
        WHEN published_at::time BETWEEN '08:00:00' AND '09:59:59' THEN 5
        WHEN published_at::time BETWEEN '10:00:00' AND '11:59:59' THEN 6
        WHEN published_at::time BETWEEN '12:00:00' AND '13:59:59' THEN 7
        WHEN published_at::time BETWEEN '14:00:00' AND '15:59:59' THEN 8
        WHEN published_at::time BETWEEN '16:00:00' AND '17:59:59' THEN 9
        WHEN published_at::time BETWEEN '18:00:00' AND '19:59:59' THEN 10
        WHEN published_at::time BETWEEN '20:00:00' AND '21:59:59' THEN 11
        WHEN published_at::time BETWEEN '22:00:00' AND '23:59:59' THEN 12
    END AS time_cat
FROM charon_data_analysis
"""
)

# ANOVA
df_8_ht
df_8_groups = [group['video_view_count'].values for name, group in df_8_ht.groupby('time_cat')]

df_8_anova = stats.f_oneway(*df_8_groups)
print(f'ANOVA results: {df_8_anova}')
# Post-hoc Test
df_8_posthoc = pairwise_tukeyhsd(
    endog=df_8_ht['video_view_count'], 
    groups=df_8_ht['time_cat'],
    alpha=0.05)
df_8_posthoc = pd.DataFrame(data=df_8_posthoc.summary().data[1:], columns=df_8_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_8_posthoc.loc[(df_8_posthoc['group1']==12) | (df_8_posthoc['group2']==12)]}")

ANOVA results: F_onewayResult(statistic=242.52115941984613, pvalue=0.0)
Post-Hoc results:
    group1  group2     meandiff  p-adj        lower        upper  reject
10       1      12  150052.3844    0.0  134946.9602  165157.8086    True
20       2      12  180730.6598    0.0  164656.5486  196804.7711    True
29       3      12  191904.7966    0.0  174243.2386  209566.3547    True
37       4      12  173240.9421    0.0  156071.1197  190410.7645    True
44       5      12  132514.7883    0.0  115371.2661  149658.3105    True
50       6      12  117185.7304    0.0  102731.9313  131639.5295    True
55       7      12   97718.7555    0.0   85786.9137  109650.5972    True
59       8      12  134909.4509    0.0  121684.7552  148134.1467    True
62       9      12  104654.1562    0.0   92635.1865  116673.1258    True
64      10      12  107176.1549    0.0   95890.0543  118462.2555    True
65      11      12  116133.0625    0.0  104686.6865  127579.4386    True


**5. What category is the most trending?**  

**Choice of Test**: ANOVA, Post-hoc (Tukey's HSD)  
ANOVA test will be used to see if there are significant differences between the video categories. The categories are the different genres that the videos are categorized as.

Then, Post-hoc Testing will be used to further analyze to determine which specific categories differ from each other and see if what can be seen from the eye that the category 'Film & Animation' holds significantly more views than other categories.

In [35]:
df_9_ht = sql_query(
"""
SELECT * FROM charon_data_analysis
"""
)
# ANOVA
df_9_groups = [group['video_view_count'].values for name, group in df_9_ht.groupby('category_name')]
df_9_anova = stats.f_oneway(*df_9_groups)
print(f'ANOVA results: {df_9_anova}')
# Post-hoc Test
df_9_posthoc = pairwise_tukeyhsd(
    endog=df_9_ht['video_view_count'], 
    groups=df_9_ht['category_name'],
    alpha=0.05)
df_9_posthoc = pd.DataFrame(data=df_9_posthoc.summary().data[1:], columns=df_9_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_9_posthoc.loc[(df_9_posthoc['group1']=='Film & Animation') | (df_9_posthoc['group2']=='Film & Animation')]}")

ANOVA results: F_onewayResult(statistic=48.93590545912353, pvalue=4.2571579370321975e-127)
Post-Hoc results:
              group1                group2     meandiff   p-adj        lower  \
3   Autos & Vehicles      Film & Animation  402324.7541  0.0000  185464.2359   
15            Comedy      Film & Animation  167484.2182  0.0000   81547.0950   
26         Education      Film & Animation  369266.2478  0.0000  260730.2492   
36     Entertainment      Film & Animation  158830.9073  0.0000   84626.3494   
46  Film & Animation                Gaming -290499.6201  0.0000 -358762.8620   
47  Film & Animation         Howto & Style -227766.8166  0.0000 -371093.9324   
48  Film & Animation                 Music  -72023.7707  0.9967 -278883.0292   
49  Film & Animation       News & Politics -357670.2541  0.5118 -866507.2243   
50  Film & Animation        People & Blogs -335630.4005  0.0000 -405843.3982   
51  Film & Animation        Pets & Animals -327179.3126  0.0000 -449486.9758   
52  Film & 

**6.1 Is the length of the video related to views? Shorts vs. Videos**  

**Choice of Test**: T-Test  
Independent samples T-Test will be used to compare the mean of the length of categories first it being shorts vs. videos. As it can be seen that shorts usually have way more views than normal videos, but to see if the differences are significant even between Charon Creators.

In [36]:
df_10_ht = sql_query(
"""
SELECT 
    *
	,CASE
		WHEN duration_seconds <= 60 THEN 'shorts'
		WHEN duration_seconds > 60 THEN 'video'
	END AS content_type
FROM charon_data_analysis
"""
)
# # Q-Q plot for shorts views
# plt.figure()
# stats.probplot(df_10_ht[df_10_ht['content_type']=='shorts']['video_view_count'], dist="norm", plot=plt)
# plt.title("Q-Q Plot for Shorts Views")
# plt.show()

# # # Q-Q plot for normal video views
# plt.figure()
# stats.probplot(df_10_ht[df_10_ht['content_type']=='video']['video_view_count'], dist="norm", plot=plt)
# plt.title("Q-Q Plot for Normal Video Views")
# plt.show()

# Perform independent samples t-test
df_10_ttest = stats.ttest_ind(
    df_10_ht[df_10_ht['content_type']=='shorts']['video_view_count'], 
    df_10_ht[df_10_ht['content_type']=='video']['video_view_count'], 
    equal_var=False)

print(f'T-Test results:\n {df_10_ttest}')

T-Test results:
 TtestResult(statistic=29.15057044008948, pvalue=2.204820326594849e-177, df=7773.2419616945435)


**6.2 Does it have a different effect in different categories?**  

**Choice of Test**: ANOVA, Post-hoc (Tukey's HSD)  
ANOVA test will be used to see if there are significant differences between the video categories in shorts and normal videos. The categories are the different genres that the videos are categorized as.

Then, Post-hoc Testing will be used to further analyze to determine which specific categories differ from each other and see if what can be seen from the eye that the category 'Film & Animation' holds significantly more views than other categories in normal videos and 'Sports' holds significantly more views than other categories in shorts.

In [37]:
# 6.2 Does it have a different effect in different categories?
df_11_ht = sql_query(
"""
SELECT 
    *
	,CASE
		WHEN duration_seconds <= 60 THEN 'shorts'
		WHEN duration_seconds > 60 THEN 'video'
	END AS content_type
FROM charon_data_analysis
"""
)
# ANOVA shorts
df_11_shorts = df_11_ht[df_11_ht['content_type'] == 'shorts']
df_11_shorts_groups = [group['video_view_count'].values for name, group in df_11_shorts.groupby('category_name')]
df_11_shorts_anova = stats.f_oneway(*df_11_shorts_groups)
print(f'ANOVA results: {df_11_shorts_anova}')
# Post-hoc Test shorts
df_11_shorts_posthoc = pairwise_tukeyhsd(
    endog=df_11_shorts['video_view_count'], 
    groups=df_11_shorts['category_name'],
    alpha=0.05)
df_11_shorts_posthoc = pd.DataFrame(data=df_11_shorts_posthoc.summary().data[1:], columns=df_11_shorts_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_11_shorts_posthoc.loc[(df_11_shorts_posthoc['group1']=='Sports') | (df_11_shorts_posthoc['group2']=='Sports')]}")


# ANOVA video
df_11_video = df_11_ht[df_11_ht['content_type'] == 'video']
df_11_video_groups = [group['video_view_count'].values for name, group in df_11_video.groupby('category_name')]
df_11_video_anova = stats.f_oneway(*df_11_video_groups)
print(f'ANOVA results: {df_11_video_anova}')
# Post-hoc Test video
df_11_video_posthoc = pairwise_tukeyhsd(
    endog=df_11_video['video_view_count'], 
    groups=df_11_video['category_name'],
    alpha=0.05)
df_11_video_posthoc = pd.DataFrame(data=df_11_video_posthoc.summary().data[1:], columns=df_11_video_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_11_video_posthoc.loc[(df_11_video_posthoc['group1']=='Film & Animation') | (df_11_video_posthoc['group2']=='Film & Animation')]}")

ANOVA results: F_onewayResult(statistic=2.0627574869175573, pvalue=0.019671781633968922)


/Users/robertopark/Documents/code-projects/datascience-projects/charon-analysis/.venv/lib/python3.11/site-packages/scipy/integrate/_quadpack_py.py:1272: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


Post-Hoc results:
                  group1           group2     meandiff   p-adj         lower  \
9                 Comedy           Sports  168955.2193  0.9991 -4.449827e+05   
19             Education           Sports  545981.7222  0.3902 -1.905887e+05   
28         Entertainment           Sports  385929.1853  0.5495 -1.875046e+05   
36      Film & Animation           Sports  389949.2321  0.5686 -1.963742e+05   
43                Gaming           Sports  358460.8872  0.6566 -2.123460e+05   
49                 Music           Sports  100550.1667  1.0000 -1.408970e+06   
54       News & Politics           Sports  594110.6667  0.8677 -5.469793e+05   
58        People & Blogs           Sports  426367.5365  0.4420 -1.671615e+05   
61        Pets & Animals           Sports  583233.9167  0.6130 -3.188769e+05   
63  Science & Technology           Sports  608686.1667  0.9770 -9.008340e+05   
65                Sports  Travel & Events -602816.0238  0.3183 -1.380338e+06   

           upper  rej

**6.3 From only normal videos (not shorts), which types of video lengths have the most views?**  

**Choice of Test**: ANOVA, Post-hoc (Tukey's HSD)  
ANOVA test will be used to see if there are significant differences between the video lengths in normal videos. The groups are divided into 'short', 'medium', and 'long' being 1~5 minutes, 5~20 minutes, and 20~ minutes long, respectively.

Then, Post-hoc Testing will be used to further analyze to determine which lengths differ statistically significant. It can be seen that medium have a much higher view than the short and long videos.

In [38]:
df_12_ht = sql_query(
"""
SELECT
    *
    ,CASE
        WHEN duration_seconds BETWEEN 60 AND 60*5 THEN 'short'
        WHEN duration_seconds BETWEEN 60*5+1 AND 60*20 THEN 'medium'
        WHEN duration_seconds > 60*20 THEN 'long'
    END AS video_length
FROM charon_data_analysis
WHERE duration_seconds > 60
"""
)
df_12_ht
# ANOVA
df_12_groups = [group['video_view_count'].values for name, group in df_12_ht.groupby('video_length')]
df_12_anova = stats.f_oneway(*df_12_groups)
print(f'ANOVA results: {df_12_anova}')
# Post-hoc Test
df_12_posthoc = pairwise_tukeyhsd(
    endog=df_12_ht['video_view_count'], 
    groups=df_12_ht['video_length'],
    alpha=0.05)
df_12_posthoc = pd.DataFrame(data=df_12_posthoc.summary().data[1:], columns=df_12_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_12_posthoc}")


ANOVA results: F_onewayResult(statistic=801.7144537452605, pvalue=0.0)
Post-Hoc results:
   group1  group2    meandiff   p-adj       lower       upper  reject
0    long  medium  64895.9017  0.0000  60890.8386  68900.9648    True
1    long   short    401.1053  0.9897  -6465.8526   7268.0632   False
2  medium   short -64494.7964  0.0000 -71245.8708 -57743.7221    True


**6.4 From only normal videos (not shorts), which types of video lengths have the most views by category?**  

**Choice of Test**: ANOVA, Post-hoc (Tukey's HSD)  
ANOVA test will be used to see if there are significant differences between categories as genres by video lengths in normal videos. The groups are divided into 'short', 'medium', and 'long' being 1~5 minutes, 5~20 minutes, and 20~ minutes long, respectively.

Then, Post-hoc Testing will be used to further analyze to determine which categories by lengths differ statistically significant. It can be seen that from short videos 'Film & Animation' and 'Comedy', from medium videos 'Film & Animation', and from long videos 'Film & Animation and 'Science & Technology' have the most have significantly more views than other categories.

In [56]:
df_13_ht = sql_query(
"""
SELECT
    *
    ,CASE
        WHEN duration_seconds BETWEEN 60 AND 60*5 THEN 'short'
        WHEN duration_seconds BETWEEN 60*5+1 AND 60*20 THEN 'medium'
        WHEN duration_seconds > 60*20 THEN 'long'
    END AS video_length
FROM charon_data_analysis
WHERE duration_seconds > 60
"""
)

# ANOVA short
df_13_short = df_13_ht[df_13_ht['video_length'] == 'short']
df_13_short_groups = [group['video_view_count'].values for name, group in df_13_short.groupby('category_name')]
df_13_short_anova = stats.f_oneway(*df_13_short_groups)
print(f'ANOVA results: {df_13_short_anova}')
# Post-hoc Test short
df_13_short_posthoc = pairwise_tukeyhsd(
    endog=df_13_short['video_view_count'], 
    groups=df_13_short['category_name'],
    alpha=0.05)
df_13_short_posthoc = pd.DataFrame(data=df_13_short_posthoc.summary().data[1:], columns=df_13_short_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_13_short_posthoc.loc[(df_13_short_posthoc['group1'].isin(('Film & Animation', 'Comedy'))) | (df_13_short_posthoc['group2'].isin(('Film & Animation', 'Comedy')))]}")

# ANOVA medium
df_13_medium = df_13_ht[df_13_ht['video_length'] == 'medium']
df_13_medium_groups = [group['video_view_count'].values for name, group in df_13_medium.groupby('category_name')]
df_13_medium_anova = stats.f_oneway(*df_13_medium_groups)
print(f'ANOVA results: {df_13_medium_anova}')
# Post-hoc Test medium
df_13_medium_posthoc = pairwise_tukeyhsd(
    endog=df_13_medium['video_view_count'], 
    groups=df_13_medium['category_name'],
    alpha=0.05)
df_13_medium_posthoc = pd.DataFrame(data=df_13_medium_posthoc.summary().data[1:], columns=df_13_medium_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_13_medium_posthoc.loc[(df_13_medium_posthoc['group1']=='Film & Animation') | (df_13_medium_posthoc['group2']=='Film & Animation')]}")

# ANOVA long
df_13_long = df_13_ht[df_13_ht['video_length'] == 'long']
df_13_long_groups = [group['video_view_count'].values for name, group in df_13_long.groupby('category_name')]
df_13_long_anova = stats.f_oneway(*df_13_long_groups)
print(f'ANOVA results: {df_13_medium_anova}')
# Post-hoc Test long
df_13_long_posthoc = pairwise_tukeyhsd(
    endog=df_13_long['video_view_count'], 
    groups=df_13_long['category_name'],
    alpha=0.05)
df_13_long_posthoc = pd.DataFrame(data=df_13_long_posthoc.summary().data[1:], columns=df_13_long_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_13_long_posthoc.loc[(df_13_long_posthoc['group1'].isin(('Film & Animation', 'Science & Technology'))) | (df_13_long_posthoc['group2'].isin(('Film & Animation', 'Science & Technology')))]}")


ANOVA results: F_onewayResult(statistic=21.13928949219221, pvalue=5.876830818059036e-43)
Post-Hoc results:
              group1                group2     meandiff   p-adj        lower  \
0             Comedy             Education -232898.0758  0.0022 -417509.9803   
1             Comedy         Entertainment  -82889.5604  0.7192 -220978.3478   
2             Comedy      Film & Animation   37121.0909  1.0000 -151461.4163   
3             Comedy                Gaming -221415.8919  0.0000 -354862.4685   
4             Comedy         Howto & Style -127284.9091  0.4046 -300509.0030   
5             Comedy                 Music -214833.2424  0.3803 -502897.7805   
6             Comedy        People & Blogs -215530.0828  0.0000 -351680.3217   
7             Comedy        Pets & Animals -219413.9880  0.0001 -370837.2681   
8             Comedy  Science & Technology -251657.9091  0.8287 -713588.8262   
9             Comedy                Sports -180025.6783  0.0533 -361209.8907   
10           

**7. Does the visuals of the creator being face, no-face, or virtual matter?**  

**Choice of Test**: ANOVA, Post-hoc (Tukey's HSD)  
ANOVA test will be used to see if there are significant differences between the visuals of the creators as in do they not show their faces, do they use virtual characters or do they use their faces.

Then, Post-hoc Testing will be used to further analyze to determine which categories by visuals differ statistically significant. It can be seen that from creators who show their faces have more views than ones that don't.

In [42]:
df_14_ht = sql_query(
"""
SELECT
	name
	,CASE
		WHEN name IN ('피닉스박', '죠니월드', '또시', '이재석', '모아요', '류제홍', '코뚱잉', '인섹', '김용녀', '돌카사', '학살', '막눈', '강인경', '한동숙', '꼴랑이', '윤가놈', '스틸로', '명훈', '랄로', '괴물쥐', '소행성612', '핑크자크', '버니버니', '신해조', '이석현') THEN 'face'
		WHEN name IN ('삐야기', '스나랑', '후즈') THEN 'virtual'
		WHEN name IN ('냄새', '링규링규링', '지명', '2수연', '전쓰트', '정동글', '프로젝트롤', '헤이스트', '른쿄', '블루위키', '아빠킹', '김데데', '도개', 'THIRD', '미남홀란드', '종원TV', '꼬예유', 'ATK', '추털이', '순규박', '모양몬', '이클리피아', '멀럭킹', '송사리', '제갈병춘', 'PAKA', '지보배', '핑맨', '흐쟁이', '미야', '고수달') THEN 'no-face'
	END AS creator_type
	,subscriber_count
	,channel_view_count
	,video_view_count
	,video_count
FROM charon_data_analysis
"""
)
# ANOVA
df_14_groups = [group['video_view_count'].values for name, group in df_14_ht.groupby('creator_type')]
df_14_anova = stats.f_oneway(*df_14_groups)
print(f'ANOVA results: {df_14_anova}')
# Post-hoc Test
df_14_posthoc = pairwise_tukeyhsd(
    endog=df_14_ht['video_view_count'], 
    groups=df_14_ht['creator_type'],
    alpha=0.05)
df_14_posthoc = pd.DataFrame(data=df_14_posthoc.summary().data[1:], columns=df_14_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_14_posthoc}")

ANOVA results: F_onewayResult(statistic=542.4139381992954, pvalue=6.443362566253018e-235)
Post-Hoc results:
    group1   group2    meandiff  p-adj       lower       upper  reject
0     face  no-face -54271.2134    0.0 -58428.4528 -50113.9740    True
1     face  virtual  18695.8877    0.0   8783.0772  28608.6983    True
2  no-face  virtual  72967.1011    0.0  63190.2850  82743.9172    True


**8. Does the length of the title show a difference in views performance?**  

**Choice of Test**: ANOVA, Post-hoc (Tukey's HSD)  
ANOVA test will be used to see if there are significant differences between the length of their video titles. The categorization of the length of video titles are Korean syllables including spaces being between 0~4, 5~20, 21~35, and 36~ for, short, medium, long, and very long respectively.

Then, Post-hoc Testing will be used to further analyze to determine which categories by title length differ statistically significant. It can be seen that shorter titles have more views than longer titles across all lengths.

In [45]:
df_15_ht = sql_query(
"""
SELECT
    *
    ,CASE
		WHEN LENGTH(title) BETWEEN 0 AND 4 THEN 'short'
		WHEN LENGTH(title) BETWEEN 5 AND 20 THEN 'medium'
		WHEN LENGTH(title) BETWEEN 21 AND 35 THEN 'long'
		WHEN LENGTH(title) >= 36 THEN 'very long'
	END AS title_length
FROM charon_data_analysis
"""
)
# ANOVA
df_15_groups = [group['video_view_count'].values for name, group in df_15_ht.groupby('title_length')]
df_15_anova = stats.f_oneway(*df_15_groups)
print(f'ANOVA results: {df_15_anova}')
# Post-hoc Test
df_15_posthoc = pairwise_tukeyhsd(
    endog=df_15_ht['video_view_count'], 
    groups=df_15_ht['title_length'],
    alpha=0.05)
df_15_posthoc = pd.DataFrame(data=df_15_posthoc.summary().data[1:], columns=df_15_posthoc.summary().data[0])
print(f"Post-Hoc results:\n{df_15_posthoc}")

ANOVA results: F_onewayResult(statistic=1596.2941641345988, pvalue=0.0)
Post-Hoc results:
   group1     group2     meandiff  p-adj        lower        upper  reject
0    long     medium  113837.9455    0.0  107802.3096  119873.5813    True
1    long      short  212279.4123    0.0  186391.4530  238167.3717    True
2    long  very long  -28825.2774    0.0  -33835.9834  -23814.5714    True
3  medium      short   98441.4669    0.0   72435.6415  124447.2923    True
4  medium  very long -142663.2228    0.0 -148251.0390 -137075.4067    True
5   short  very long -241104.6897    0.0 -266891.9197 -215317.4597    True


# Data Visualization

The visualization part will be done in Tableau and the csv files required for the visualizations in Tableau will be made via PostgreSQL's pgAdmin4 and saved into the directory 'data/tableau'. The dashboards will be in my Tableau Public: [link](https://public.tableau.com/app/profile/roberto.parkr/vizzes)